In [2]:
import MetaTrader5 as mt5
import schedule
import numpy as np
import pandas as pd
from datetime import datetime
from  backtest import Backtester,  get_ohlc_history, create_price_fig, evaluate_backtest

In [3]:
# Conecte mt5
mt5.initialize()
# logine mt5
login = 52185665
password = '3cAJz$AQiKwMq0'
server = 'ICMarketsSC-Demo'

mt5.login(login,password,server)

True

In [4]:
# this function retreives olhc data from mt5 account and return a data frame
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    df = pd.DataFrame(ohlc)
    df['time'] = pd.to_datetime(df['time'], unit= 's')#.dt.tz_localize('UTC')
    return df

In [5]:
# 1 minut df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_M1
start_datetime = datetime(2025,8,1)
end_datetime = datetime.now()
df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
df

,time,open,high,low,close,tick_volume,spread,real_volume
0,2025-07-31 23:00:00,1.14125,1.14133,1.14102,1.14105,92,0,0
1,2025-07-31 23:01:00,1.14105,1.14117,1.14101,1.14111,73,0,0
2,2025-07-31 23:02:00,1.14111,1.14124,1.14098,1.14112,89,0,0
3,2025-07-31 23:03:00,1.14112,1.14145,1.14112,1.14143,52,0,0
4,2025-07-31 23:04:00,1.14145,1.14158,1.14142,1.14152,64,0,0
...,...,...,...,...,...,...,...,...
66224,2025-10-03 23:52:00,1.17394,1.17398,1.17391,1.17396,25,5,0
66225,2025-10-03 23:53:00,1.17396,1.17396,1.17386,1.17389,24,5,0
66226,2025-10-03 23:54:00,1.17389,1.17397,1.17387,1.17397,29,5,0
66227,2025-10-03 23:55:00,1.17396,1.17399,1.17394,1.17394,29,6,0


In [ ]:
#df["spread"] = df["spread"] / 1000
df["atr"] = df["high"].rolling(14).max() - df["low"].rolling(14).min()
df["vwap"] = (df["close"] * df["tick_volume"]).cumsum() / df["tick_volume"].cumsum()
df["volume_spike"] = df["tick_volume"] > df["tick_volume"].rolling(10).mean() * 1.5

# === Signal Generation ===
def generate_signals(df):
    signals = []
    for i in range(14, len(df)):
        row = df.iloc[i]
        prev = df.iloc[i-1]

        cond1 = row["close"] < row["vwap"] - 2 * row["atr"]
        cond2 = row["spread"] < 0.0002
        cond3 = row["volume_spike"]
        cond4 = row["close"] > row["open"] and prev["close"] < prev["open"]

        if cond1 and cond2 and cond3 and cond4: 
            signals.append(i)
    return signals

signals = generate_signals(df)

# === Backtest ===

results = []
rr = 3  # risk-reward ratio

for i in signals:
    entry_price = df.iloc[i]["close"]
    sl = entry_price - 0.5 * df.iloc[i]["atr"]
    tp = entry_price + rr * (entry_price - sl)

    for j in range(i+1, min(i+60, len(df))):  # look ahead for next ~2 hours
        high = df.iloc[j]["high"]
        low = df.iloc[j]["low"]
        if low <= sl:
            results.append(("loss", entry_price, sl))
            break
        elif high >= tp:
            results.append(("win", entry_price, tp))
            break
    else:
        # time-based exit
        close_exit = df.iloc[i+60]["close"]
        if close_exit > entry_price:
            results.append(("win", entry_price, close_exit))
        else:
            results.append(("loss", entry_price, close_exit))
# === Metrics ===
wins = [r for r in results if r[0] == "win"]
losses = [r for r in results if r[0] == "loss"]
win_rate = len(wins) / len(results)
r_r_ratio = np.mean([abs(w[2]-w[1]) for w in wins]) / np.mean([abs(l[2]-l[1]) for l in losses])


entry_price = df.iloc[i]["close"]
df['sl'] = entry_price - 0.5 * df.iloc[i]["atr"]
df['tp'] = entry_price + rr * (entry_price - sl)

print(f"Total trades: {len(results)}")
print(f"Win rate: {win_rate:.2%}")
print(f"Risk-Reward Ratio: {r_r_ratio:.2f}")
print(results)


Total trades: 41
Win rate: 21.95%
Risk-Reward Ratio: 3.01
[('win', np.float64(1.16051), np.float64(1.1612599999999995)), ('loss', np.float64(1.1611), np.float64(1.16093)), ('loss', np.float64(1.16049), np.float64(1.1602350000000001)), ('loss', np.float64(1.16054), np.float64(1.16045)), ('win', np.float64(1.1604700000000001), np.float64(1.1607400000000005)), ('loss', np.float64(1.16122), np.float64(1.161125)), ('loss', np.float64(1.16111), np.float64(1.1609350000000003)), ('loss', np.float64(1.16126), np.float64(1.161115)), ('loss', np.float64(1.16133), np.float64(1.16112)), ('loss', np.float64(1.16078), np.float64(1.1604999999999999)), ('loss', np.float64(1.16055), np.float64(1.1602700000000001)), ('loss', np.float64(1.15875), np.float64(1.1585649999999998)), ('loss', np.float64(1.15862), np.float64(1.1584349999999999)), ('win', np.float64(1.15876), np.float64(1.15979)), ('loss', np.float64(1.15936), np.float64(1.159145)), ('win', np.float64(1.16105), np.float64(1.161575)), ('loss', np

In [7]:
prev = df.iloc[i-1]
def entring_stoploss_profit(row) :
    hour = row['time'].hour
    #idx = row.name
    #if idx == 0:
    #    return None
    #prev = df.iloc[idx - 1]

    if (
        row["close"] < row["vwap"] - 2 * row["atr"]
        and row["spread"] < 0.0002
        and row["volume_spike"]
        and row["close"] > row["open"]
        and prev["close"] < prev["open"]
        #and (hour < 11 or hour > 15)
    ):
        return "buy"
    return None
df["signal"] = df.apply(entring_stoploss_profit, axis=1)
df[["signal","time"]].dropna()

,signal,time
21378,buy,2025-08-21 19:39:00
21472,buy,2025-08-21 21:13:00
21501,buy,2025-08-21 21:42:00
21507,buy,2025-08-21 21:48:00
21516,buy,2025-08-21 21:57:00
...,...,...
57782,buy,2025-09-26 03:03:00
57791,buy,2025-09-26 03:12:00
57792,buy,2025-09-26 03:13:00
57866,buy,2025-09-26 04:27:00


In [8]:
def excution():
    tick = mt5.symbol_info_tick(symbol)
    lot = 10000
    price = tick.ask
    buy_order_type = mt5.ORDER_TYPE_BUY
    if df["signal"].iloc[-1] not in ["buy", "sell"]:
        print("No trade signal (neutral) — skipping trade.")
        return
    if df["low"] <= df["sl"] or df["high"] >= df["tp"]:
            buy_sl = df["close"].iloc[-1]
    if df["signal"].iloc[-1] == "buy" and not buy_order_type.shape[0] :
        request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": buy_order_type,
        "price": price,
        "sl": buy_sl,
        #"tp": tp,
        "deviation": 10,
        "magic": 234000,
        "comment": f"buy executed {price}",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
        result = mt5.order_send(request)

        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}")
        else:
            print(f"buy trade placed successfully at {price}")
            print(f"SL: {buy_sl}")
        
excution()

No trade signal (neutral) — skipping trade.


In [9]:

# create trade logic
def on_bar(data, trades, orders):
    open_trades = trades[trades['state'] == 'open']
    num_open_trades = open_trades.shape[0]
    account_balance = 10000  # your account in USD
    risk_percent = 0.20  # 2%
    pip_value_per_lot = 10  # on EURUSD, 1 lot = $10 per pip
    risk_amount = account_balance * risk_percent
    # entry signal
    if data['signal'] == 'buy' :#and not num_open_trades:
        volume = 500000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'buy')
    
    elif data['signal'] == 'sell' and not num_open_trades:
        volume = 500000 #risk_amount / (stop_loss_pips * pip_value_per_lot)
        orders.open_trade(symbol, volume, 'sell')
        
# exit signal
    if num_open_trades:
        trade = open_trades.iloc[0]

        if trade['order_type'] == 'buy' and data["low"] <= data["sl"] or data["high"] >= data["tp"]: 
            orders.close_trade(trade)
        elif trade['order_type'] == 'sell'  :
            orders.close_trade(trade)

In [10]:
# backtest parameters
starting_balance = 10000
currency = 'USD'
exchange_rate = 1
commission = -7 / 500000

# backtest
bt = Backtester()
bt.set_starting_balance(starting_balance, currency=currency)
bt.set_exchange_rate(exchange_rate)
bt.set_commission(commission)

bt.set_historical_data(df)
bt.set_on_bar(on_bar)
#get_ohlc_history(symbol, timeframe, start_datetime, end_datetime)
bt.run_backtest()

bt.trades


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,info,profit,commission,profit_net,profit_cumulative,balance
0,closed,EURUSD,buy,500000,2025-08-21 19:39:00,1.16036,2025-08-21 19:40:00,1.16047,0,0,{},55.0,-7.0,48.0,48.0,10048.0
1,closed,EURUSD,buy,500000,2025-08-21 21:13:00,1.16106,2025-08-21 21:14:00,1.1611,0,0,{},20.0,-7.0,13.0,61.0,10061.0
2,closed,EURUSD,buy,500000,2025-08-21 21:42:00,1.16109,2025-08-21 21:43:00,1.1611,0,0,{},5.0,-7.0,-2.0,59.0,10059.0
3,closed,EURUSD,buy,500000,2025-08-21 21:48:00,1.16078,2025-08-21 21:49:00,1.1608,0,0,{},10.0,-7.0,3.0,62.0,10062.0
4,closed,EURUSD,buy,500000,2025-08-21 21:57:00,1.16098,2025-08-21 21:58:00,1.16103,0,0,{},25.0,-7.0,18.0,80.0,10080.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,closed,EURUSD,buy,500000,2025-09-26 03:03:00,1.16596,2025-09-26 03:04:00,1.16614,0,0,{},90.0,-7.0,83.0,3034.0,13034.0
83,closed,EURUSD,buy,500000,2025-09-26 03:12:00,1.16592,2025-09-26 03:13:00,1.16598,0,0,{},30.0,-7.0,23.0,3057.0,13057.0
84,closed,EURUSD,buy,500000,2025-09-26 03:13:00,1.16598,2025-09-26 03:14:00,1.16614,0,0,{},80.0,-7.0,73.0,3130.0,13130.0
85,closed,EURUSD,buy,500000,2025-09-26 04:27:00,1.16709,2025-09-26 04:38:00,1.16698,0,0,{},-55.0,-7.0,-62.0,3068.0,13068.0


In [11]:
mm = bt.plot_balance()
display(mm)

In [12]:
evaluate_backtest(bt.trades)

Daily Drawdown : -175.00
Max Drawdown (portfolio): -175.00
worst trades (trade MAE): [np.float64(-14.999999999987246), np.float64(-5.000000000032756), np.float64(-4.999999999921734), np.float64(-44.99999999996174), np.float64(-4.999999999921734), np.float64(-35.00000000000725), np.float64(-10.000000000065512), np.float64(-25.000000000052758), np.float64(-9.99999999995449), np.float64(-9.99999999995449), np.float64(-5.000000000032756), np.float64(0.0), np.float64(0.0), np.float64(-20.000000000020002), np.float64(-14.999999999987246), np.float64(-5.000000000032756), np.float64(0.0), np.float64(-5.000000000032756), np.float64(-19.99999999990898), np.float64(-9.99999999995449), np.float64(0.0), np.float64(-10.000000000065512), np.float64(-5.000000000032756), np.float64(0.0), np.float64(-34.999999999896225), np.float64(-10.000000000065512), np.float64(-5.000000000032756), np.float64(-35.00000000000725), np.float64(-5.000000000032756), np.float64(0.0), np.float64(0.0), np.float64(-30.0000000

,order_type,profit
0,buy,3585.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,profit,commission,profit_net,profit_cumulative,balance,date,time,intrabar_drawdown,current_max,drawdown1
0,closed,EURUSD,buy,500000,2025-08-21 19:39:00,1.16036,2025-08-21 19:40:00,1.16047,0,0,...,55.0,-7.0,48.0,48.0,10048.0,2025-08-21,2025-08-21 19:40:00,-15.0,48.0,0.0
1,closed,EURUSD,buy,500000,2025-08-21 21:13:00,1.16106,2025-08-21 21:14:00,1.1611,0,0,...,20.0,-7.0,13.0,61.0,10061.0,2025-08-21,2025-08-21 21:14:00,-5.0,61.0,0.0
2,closed,EURUSD,buy,500000,2025-08-21 21:42:00,1.16109,2025-08-21 21:43:00,1.1611,0,0,...,5.0,-7.0,-2.0,59.0,10059.0,2025-08-21,2025-08-21 21:43:00,-5.0,61.0,-2.0
3,closed,EURUSD,buy,500000,2025-08-21 21:48:00,1.16078,2025-08-21 21:49:00,1.1608,0,0,...,10.0,-7.0,3.0,62.0,10062.0,2025-08-21,2025-08-21 21:49:00,-45.0,62.0,0.0
4,closed,EURUSD,buy,500000,2025-08-21 21:57:00,1.16098,2025-08-21 21:58:00,1.16103,0,0,...,25.0,-7.0,18.0,80.0,10080.0,2025-08-21,2025-08-21 21:58:00,-5.0,80.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,closed,EURUSD,buy,500000,2025-09-26 02:25:00,1.16613,2025-09-26 02:26:00,1.16622,0,0,...,45.0,-7.0,38.0,2948.0,12948.0,2025-09-26,2025-09-26 02:26:00,-70.0,2948.0,0.0
81,closed,EURUSD,buy,500000,2025-09-26 03:01:00,1.16608,2025-09-26 03:02:00,1.1661,0,0,...,10.0,-7.0,3.0,2951.0,12951.0,2025-09-26,2025-09-26 03:02:00,-75.0,2951.0,0.0
82,closed,EURUSD,buy,500000,2025-09-26 03:03:00,1.16596,2025-09-26 03:04:00,1.16614,0,0,...,90.0,-7.0,83.0,3034.0,13034.0,2025-09-26,2025-09-26 03:04:00,0.0,3034.0,0.0
83,closed,EURUSD,buy,500000,2025-09-26 03:12:00,1.16592,2025-09-26 03:13:00,1.16598,0,0,...,30.0,-7.0,23.0,3057.0,13057.0,2025-09-26,2025-09-26 03:13:00,-10.0,3057.0,0.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,profit,commission,profit_net,profit_cumulative,balance,date,time,intrabar_drawdown,current_max,drawdown1
85,closed,EURUSD,buy,500000,2025-09-26 04:27:00,1.16709,2025-09-26 04:38:00,1.16698,0,0,...,-55.0,-7.0,-62.0,3068.0,13068.0,2025-09-26,2025-09-26 04:38:00,-65.0,3130.0,-62.0
86,closed,EURUSD,buy,500000,2025-09-26 04:32:00,1.16718,2025-09-26 04:42:00,1.16701,0,0,...,-85.0,-7.0,-92.0,2976.0,12976.0,2025-09-26,2025-09-26 04:42:00,-120.0,3130.0,-154.0


,state,symbol,order_type,volume,open_time,open_price,close_time,close_price,sl,tp,...,date,time,intrabar_drawdown,current_max,drawdown1,dayofweek,hourofday,month,year,drawdown
0,closed,EURUSD,buy,500000,2025-08-21 19:39:00,1.16036,2025-08-21 19:40:00,1.16047,0,0,...,2025-08-21,2025-08-21 19:40:00,-15.0,48.0,0.0,3,19,2025-08,2025,0.0
1,closed,EURUSD,buy,500000,2025-08-21 21:13:00,1.16106,2025-08-21 21:14:00,1.1611,0,0,...,2025-08-21,2025-08-21 21:14:00,-5.0,61.0,0.0,3,21,2025-08,2025,0.0
2,closed,EURUSD,buy,500000,2025-08-21 21:42:00,1.16109,2025-08-21 21:43:00,1.1611,0,0,...,2025-08-21,2025-08-21 21:43:00,-5.0,61.0,-2.0,3,21,2025-08,2025,-2.0
3,closed,EURUSD,buy,500000,2025-08-21 21:48:00,1.16078,2025-08-21 21:49:00,1.1608,0,0,...,2025-08-21,2025-08-21 21:49:00,-45.0,62.0,0.0,3,21,2025-08,2025,0.0
4,closed,EURUSD,buy,500000,2025-08-21 21:57:00,1.16098,2025-08-21 21:58:00,1.16103,0,0,...,2025-08-21,2025-08-21 21:58:00,-5.0,80.0,0.0,3,21,2025-08,2025,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,closed,EURUSD,buy,500000,2025-09-26 03:03:00,1.16596,2025-09-26 03:04:00,1.16614,0,0,...,2025-09-26,2025-09-26 03:04:00,0.0,3034.0,0.0,4,3,2025-09,2025,0.0
83,closed,EURUSD,buy,500000,2025-09-26 03:12:00,1.16592,2025-09-26 03:13:00,1.16598,0,0,...,2025-09-26,2025-09-26 03:13:00,-10.0,3057.0,0.0,4,3,2025-09,2025,0.0
84,closed,EURUSD,buy,500000,2025-09-26 03:13:00,1.16598,2025-09-26 03:14:00,1.16614,0,0,...,2025-09-26,2025-09-26 03:14:00,-5.0,3130.0,0.0,4,3,2025-09,2025,0.0
85,closed,EURUSD,buy,500000,2025-09-26 04:27:00,1.16709,2025-09-26 04:38:00,1.16698,0,0,...,2025-09-26,2025-09-26 04:38:00,-65.0,3130.0,-62.0,4,4,2025-09,2025,-62.0


max_drawdown -154.0


ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['state', 'symbol', 'order_type', 'volume', 'open_time', 'open_price', 'close_time', 'close_price', 'sl', 'tp', 'info', 'profit', 'commission', 'profit_net', 'profit_cumulative', 'balance', 'date', 'time', 'intrabar_drawdown', 'current_max', 'drawdown1', 'dayofweek', 'hourofday', 'month', 'year', 'drawdown'] but received: daily_dd